In [28]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler # Normalization - Standardization
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from imblearn.pipeline import Pipeline

In [29]:
us = pd.read_csv('../test/users_test.csv')
dev = pd.read_csv('../test/devices_test.csv')

# tr1 = pd.read_csv('../data/transactions_1.csv')
# tr2 = pd.read_csv('../data/transactions_2.csv')
# tr3 = pd.read_csv('../data/transactions_3.csv')

trans = pd.read_csv('../test/transactions_test.csv') # pd.concat([tr1, tr2, tr3])
notif = pd.read_csv('../test/notifications_test.csv')

In [30]:
us_dev = pd.merge(us, dev, on='user_id', how='inner')
us_dev['age'] = 2020 - us_dev['birth_year']
us_dev['brand'] = us_dev['brand'].str.replace('Unknown', 'Android')

brand_dict = {'Android': 0, 'Apple': 1}
us_dev['brand'] = us_dev['brand'].map(brand_dict)
# us_dev['brand'].value_counts()

# brand_dict1 = {'STANDARD': 0, 'SILVER': 1, 'GOLD': 1}
# us_dev['plan'] = us_dev['plan'].map(brand_dict1)
# us_dev['brand'].value_counts()
us_dev.head(5)
us_dev.shape
# userdf['brand'] = userdf['brand'].str.replace('Unknown', 'Android')
# us_dev.country.value_counts()

(3886, 13)

In [31]:
def change_country(x):     
  if x=='AT' or x=='BE' or x=='DE' or x=='LU' or x=='NL' or x=='LI' or x=='CH':
    return 'Central_Rich_Europe'
  elif x=='IE' or x=='GB' or x=='GG' or x=='IM' or x=='JE' or x=='GI' or x=='DK' or x=='NO' or x=='SE' or x=='FI' or x=='IS':
    return 'North_Europe_Scand'
  elif x=='FR' or x=='RE' or x=='GP' or x=='MQ' or x=='MT' or x=='IT' or x=='CY' or x=='ES' or x=='PT' or x=='GF':
    return 'Mediteranian_Europe'
  elif x=='EE' or x=='SK' or x=='HU' or x=='LV' or x=='SI' or x=='BG' or x=='GR' or x=='HR' or x=='LT' or x=='RO' or x=='PL' or x=='CZ':
    return 'East_Europe_Balkans'
  elif x=='AU':
    return 'Australia'

In [32]:
us_dev['country'] = us_dev['country'].apply(change_country)
us_dev = pd.concat([pd.get_dummies(us_dev['country']), us_dev], axis=1)
us_dev.head(5)

,Australia,Central_Rich_Europe,East_Europe_Balkans,Mediteranian_Europe,North_Europe_Scand,user_id,birth_year,country,city,created_date,user_settings_crypto_unlocked,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand,age
0,0,0,1,0,0,user_2,1987,East_Europe_Balkans,Poznań,2018-01-18 19:17:31.229096,0,0.0,0.0,21,0,0,0,33
1,0,0,0,0,1,user_8,1981,North_Europe_Scand,Fife,2018-01-03 07:28:59.063740,1,NaN,NaN,0,0,0,0,39
2,0,0,0,1,0,user_10,1980,Mediteranian_Europe,Paris,2018-01-04 08:05:15.466282,0,1.0,1.0,38,0,0,1,40
3,0,0,1,0,0,user_13,1995,East_Europe_Balkans,Anykščiai,2018-01-28 16:56:16.221271,0,1.0,1.0,10,0,0,0,25
4,0,0,0,0,1,user_17,1996,North_Europe_Scand,Birmingham,2018-01-08 04:18:47.595266,1,NaN,NaN,0,0,0,0,24


In [33]:
notif.reason.value_counts()
print(dev.user_id.shape)
print(us.user_id.shape)
print(us_dev.user_id.shape)
us_dev.isna().sum()

(3886,)
(3886,)
(3886,)


Australia                                      0
Central_Rich_Europe                            0
East_Europe_Balkans                            0
Mediteranian_Europe                            0
North_Europe_Scand                             0
user_id                                        0
birth_year                                     0
country                                        0
city                                           0
created_date                                   0
user_settings_crypto_unlocked                  0
attributes_notifications_marketing_push     1350
attributes_notifications_marketing_email    1350
num_contacts                                   0
num_referrals                                  0
num_successful_referrals                       0
brand                                          0
age                                            0
dtype: int64

In [34]:
trans = trans[(trans.transactions_state == 'COMPLETED') & (trans.amount_usd>0)]
trans['created_date'] = pd.to_datetime(trans['created_date'])
trans.transactions_type.value_counts()

CARD_PAYMENT    258042
TRANSFER         96661
TOPUP            61128
EXCHANGE         34338
ATM              16306
CARD_REFUND       2751
TAX                552
REFUND             300
Name: transactions_type, dtype: int64

In [35]:
trans_grouped = trans.groupby(['user_id'], as_index=True)['amount_usd'].agg(['sum','count','mean'])
trans_grouped.rename(columns={'sum':'total_amount', 'count':'number_transactions','mean':'avg_amount'}, inplace=True)
# trans_grouped.info()


trans_grouped2 = trans.groupby(['user_id'], as_index=True)['created_date'].agg(['min','max'])
trans_grouped2.rename(columns={'min':'first_transaction', 'max':'last_transaction'}, inplace=True)
# trans_grouped2.info()


trans_grouped2['transaction_period']=( (trans_grouped2['last_transaction']-trans_grouped2['first_transaction']) ).dt.days
# print(trans_grouped2.head(4))
# trans_grouped2['transaction_period']=trans_grouped2['transaction_period'].dt.days
#['transaction_period'].days
# print(trans_grouped2.head(4))

trans_grouped_all = pd.merge(trans_grouped, trans_grouped2, on='user_id', how='inner')
trans_grouped_all.head(5)

,total_amount,number_transactions,avg_amount,first_transaction,last_transaction,transaction_period
user_id,,,,,,
user_10,24493.07,843,29.054650,2018-01-04 08:09:47.823282,2019-05-11 11:01:00.564282,492
user_10000,12231.77,113,108.245752,2018-08-13 20:11:55.872653,2019-05-10 15:16:59.438653,269
user_10001,2428.40,27,89.940741,2018-08-12 16:19:54.595429,2019-02-03 02:02:21.980429,174
user_10006,639.92,35,18.283429,2018-09-06 18:40:41.551957,2018-12-22 18:41:18.646957,107
user_1001,1294.05,30,43.135000,2018-01-11 10:05:11.042365,2018-02-24 08:00:10.829365,43


In [36]:
trans_grouped.shape

(3701, 3)

In [37]:
data = pd.merge(us_dev, trans_grouped_all, on='user_id', how='left')


In [38]:
data.shape

(3886, 24)

In [39]:
# data[['created_date', 'first_transaction', 'last_transaction']] = pd.to_datetime(data[['created_date', 'first_transaction', 'last_transaction']]) 
data['created_date'] = pd.to_datetime(data['created_date'])
data['first_transaction'] = pd.to_datetime(data['first_transaction'])
data['last_transaction'] = pd.to_datetime(data['last_transaction'])

last_trans = data['last_transaction'].max()

In [40]:
data['user_active_time'] = (last_trans - data['created_date']).dt.days
data['user_trans_time'] = (last_trans - data['first_transaction']).dt.days
data['user_trans_periodicity'] = ( data['user_trans_time'] / data['number_transactions'] ) / data['user_active_time']
data['recency'] = (last_trans - data['last_transaction']).dt.days

data['avg_amount'] = data['total_amount'] / data['user_active_time']

print(min(data.user_trans_periodicity))
data.head(2)

0.00025786487880350697


,Australia,Central_Rich_Europe,East_Europe_Balkans,Mediteranian_Europe,North_Europe_Scand,user_id,birth_year,country,city,created_date,...,total_amount,number_transactions,avg_amount,first_transaction,last_transaction,transaction_period,user_active_time,user_trans_time,user_trans_periodicity,recency
0,0,0,1,0,0,user_2,1987,East_Europe_Balkans,Poznań,2018-01-18 19:17:31.229096,...,1036.54,81.0,2.154969,2018-01-18 19:19:56.106096,2019-04-30 05:17:13.003096,466.0,481,481.0,0.012346,15.0
1,0,0,0,0,1,user_8,1981,North_Europe_Scand,Fife,2018-01-03 07:28:59.063740,...,338.09,15.0,0.680262,2018-01-05 09:54:43.150740,2018-05-18 09:26:06.108740,132.0,497,495.0,0.066398,362.0


In [41]:
tr1 = pd.concat([pd.get_dummies(trans['transactions_type']), trans], axis=1)
tr1.head(2)

,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,REFUND,TAX,TOPUP,TRANSFER,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date
0,0,1,0,0,0,0,0,0,transaction_22,CARD_PAYMENT,AED,68.19,COMPLETED,FALSE,5691.0,Dubai,ARE,OUTBOUND,user_7969,2018-08-01 17:58:49.222557
1,0,1,0,0,0,0,0,0,transaction_24,CARD_PAYMENT,AED,13.43,COMPLETED,FALSE,5814.0,Dubai,ARE,OUTBOUND,user_7969,2018-08-02 20:33:57.365557


In [42]:
tr2 = tr1.groupby(['user_id'], as_index=True)['ATM', 'CARD_PAYMENT', 'CARD_REFUND', 'EXCHANGE', 'REFUND', 'TAX', 'TOPUP', 'TRANSFER'].agg('sum').astype(int) 
tr2.head(2)


,ATM,CARD_PAYMENT,CARD_REFUND,EXCHANGE,REFUND,TAX,TOPUP,TRANSFER
user_id,,,,,,,,
user_10,28,366,3,0,1,0,115,330
user_10000,1,63,2,18,0,0,22,7


In [43]:
def change_currency(cur):
  if cur == 'CHF':
    return 'CHF'
  elif cur == 'GBP':
    return 'GBP'
  elif cur == 'EUR':
    return 'EUR'
  else:
    return 'OTHER'

In [44]:
trans['transactions_currency'] = trans['transactions_currency'].apply(change_currency)
print(trans['transactions_currency'].value_counts())

trans = pd.concat([pd.get_dummies(trans['transactions_currency']), trans], axis=1)

trans_grouped_currency = trans.groupby(['user_id'])['CHF','EUR','GBP','OTHER'].agg('sum')

df = pd.merge(us, trans_grouped_currency, on='user_id', how='inner')

# df.info()

EUR      191163
GBP      159417
OTHER    112883
CHF        6615
Name: transactions_currency, dtype: int64


In [45]:
kk = df[['CHF', 'EUR', 'GBP', 'OTHER', 'user_id']]
kk.head(2)

,CHF,EUR,GBP,OTHER,user_id
0,0.0,4.0,10.0,67.0,user_2
1,0.0,0.0,12.0,3.0,user_8


In [46]:
data = pd.merge(data, tr2, on='user_id', how='left')
data = pd.merge(data, kk, on='user_id', how='left')

In [47]:
data.shape

(3886, 40)

In [48]:
notif = notif[notif.status == 'SENT']
nn1 = pd.concat([pd.get_dummies(notif['reason']), notif], axis=1)
nn1.head(2)

,BLACK_FRIDAY,BLUE_TUESDAY,ENGAGEMENT_SPLIT_BILL_RESTAURANT,INVEST_IN_GOLD,JOINING_ANNIVERSARY,LOST_CARD_ORDER,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,METAL_RESERVE_PLAN,NO_INITIAL_CARD_ORDER,NO_INITIAL_CARD_USE,ONBOARDING_TIPS_ACTIVATED_USERS,PROMO,PROMO_CARD_ORDER,REENGAGEMENT_ACTIVE_FUNDS,WELCOME_BACK,reason,channel,status,user_id,created_date
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1408,2018-12-11 02:04:46.284683
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,REENGAGEMENT_ACTIVE_FUNDS,PUSH,SENT,user_1295,2018-12-15 04:12:57.003255


In [49]:
nn2 = nn1.groupby(['user_id'], as_index=True)['BLACK_FRIDAY', 'BLUE_TUESDAY', 'ENGAGEMENT_SPLIT_BILL_RESTAURANT', 'INVEST_IN_GOLD',
                                              'JOINING_ANNIVERSARY', 'LOST_CARD_ORDER', 'MADE_MONEY_REQUEST_NOT_SPLIT_BILL',
                                              'METAL_RESERVE_PLAN', 'NO_INITIAL_CARD_ORDER', 'NO_INITIAL_CARD_USE',
                                              'ONBOARDING_TIPS_ACTIVATED_USERS', 'PROMO', 'PROMO_CARD_ORDER', 'REENGAGEMENT_ACTIVE_FUNDS',
                                              'WELCOME_BACK'].agg('sum').astype(int) 
nn2.head(5)


,BLACK_FRIDAY,BLUE_TUESDAY,ENGAGEMENT_SPLIT_BILL_RESTAURANT,INVEST_IN_GOLD,JOINING_ANNIVERSARY,LOST_CARD_ORDER,MADE_MONEY_REQUEST_NOT_SPLIT_BILL,METAL_RESERVE_PLAN,NO_INITIAL_CARD_ORDER,NO_INITIAL_CARD_USE,ONBOARDING_TIPS_ACTIVATED_USERS,PROMO,PROMO_CARD_ORDER,REENGAGEMENT_ACTIVE_FUNDS,WELCOME_BACK
user_id,,,,,,,,,,,,,,,
user_10,1,0,0,1,1,0,0,0,0,0,0,3,0,0,0
user_10000,0,0,0,1,0,0,0,0,0,0,0,2,0,2,1
user_10001,0,0,0,1,0,0,0,0,0,1,4,2,0,2,0
user_10006,0,0,0,0,0,0,0,0,1,0,0,1,0,3,0
user_1001,0,0,0,1,0,0,0,0,0,0,0,0,0,3,0


In [50]:
data = pd.merge(data, nn2, on='user_id', how='left')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3886 entries, 0 to 3885
Data columns (total 55 columns):
Australia                                   3886 non-null uint8
Central_Rich_Europe                         3886 non-null uint8
East_Europe_Balkans                         3886 non-null uint8
Mediteranian_Europe                         3886 non-null uint8
North_Europe_Scand                          3886 non-null uint8
user_id                                     3886 non-null object
birth_year                                  3886 non-null int64
country                                     3886 non-null object
city                                        3886 non-null object
created_date                                3886 non-null datetime64[ns]
user_settings_crypto_unlocked               3886 non-null int64
attributes_notifications_marketing_push     2536 non-null float64
attributes_notifications_marketing_email    2536 non-null float64
num_contacts                                3

In [51]:
data.isna().sum()


Australia                                      0
Central_Rich_Europe                            0
East_Europe_Balkans                            0
Mediteranian_Europe                            0
North_Europe_Scand                             0
user_id                                        0
birth_year                                     0
country                                        0
city                                           0
created_date                                   0
user_settings_crypto_unlocked                  0
attributes_notifications_marketing_push     1350
attributes_notifications_marketing_email    1350
num_contacts                                   0
num_referrals                                  0
num_successful_referrals                       0
brand                                          0
age                                            0
total_amount                                 185
number_transactions                          185
avg_amount          

In [52]:
print(data.shape)
print(us.shape)

(3886, 55)
(3886, 11)


In [53]:
data = data.fillna(0)
data.isna().sum()

Australia                                   0
Central_Rich_Europe                         0
East_Europe_Balkans                         0
Mediteranian_Europe                         0
North_Europe_Scand                          0
user_id                                     0
birth_year                                  0
country                                     0
city                                        0
created_date                                0
user_settings_crypto_unlocked               0
attributes_notifications_marketing_push     0
attributes_notifications_marketing_email    0
num_contacts                                0
num_referrals                               0
num_successful_referrals                    0
brand                                       0
age                                         0
total_amount                                0
number_transactions                         0
avg_amount                                  0
first_transaction                 

In [54]:
data.to_csv('../processed_data/testing_data.csv')